In [1]:
import math
from scipy.stats import norm
from scipy.integrate import quad
import numpy as np
# pip install scipy
# pip install numpy
# pip install -U ipykernel


# Black Scholes method

In [1]:
def black_scholes(S, K, T, r, sigma, option_type="call"):
    """
    Calcule le prix d'une option européenne à l'aide du modèle de Black-Scholes.

    :param S: Prix actuel de l'actif sous-jacent (spot price)
    :param K: Prix d'exercice de l'option (strike price)
    :param T: Temps jusqu'à l'échéance (en années)
    :param r: Taux d'intérêt sans risque (annuel, en décimales)
    :param sigma: Volatilité de l'actif sous-jacent (annuelle, en décimales)
    :param option_type: "call" pour un call, "put" pour un put
    :return: Prix de l'option
    """
    # Calcul des paramètres d1 et d2
    d1 = (math.log(S / K) + (r + (sigma**2) / 2) * T) / (sigma * math.sqrt(T))
    d2 = d1 - sigma * math.sqrt(T)
    
    if option_type == "call":
        # Prix du call
        price = S * norm.cdf(d1) - K * math.exp(-r * T) * norm.cdf(d2)
    elif option_type == "put":
        # Prix du put
        price = K * math.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    else:
        raise ValueError("option_type doit être 'call' ou 'put'")
    
    return price

In [4]:
# Exemple d'utilisation de la fonction black_scholes
S0 = 100  # Prix actuel de l'actif sous-jacent
K = 100  # Prix d'exercice
T = 1    # Temps jusqu'à l'échéance en années
r = 0.05 # Taux d'intérêt sans risque (5%)
sigma = 0.2  # Volatilité (20%)

call_price = black_scholes(S0, K, T, r, sigma, option_type="call")
put_price = black_scholes(S0, K, T, r, sigma, option_type="put")

print(f"Prix du call : {call_price:.2f}")
print(f"Prix du put : {put_price:.2f}")

Prix du call : 10.45
Prix du put : 5.57


In [ ]:
def implied_volatility(option_price, S, K, T, r, option_type="call", tol=1e-8, max_iter=100):
    """
    Calcule la volatilité implicite en utilisant une méthode itérative basée sur Newton-Raphson.
    """
    sigma = 0.2  # Initialisation avec une volatilité de départ
    for i in range(max_iter):
        price = black_scholes(S, K, T, r, sigma, option_type)
        vega_value = numerical_vega(S, K, T, r, sigma, option_type)
        
        # Mise à jour de sigma selon Newton-Raphson
        sigma_new = sigma - (price - option_price) / vega_value
        
        # Vérification du critère d'arrêt
        if abs(price - option_price) < tol:
            return sigma, i # Retourner sigma si convergence atteinte
        
        sigma = sigma_new
    # affiche le nombre d'itérations
    
    # Si la convergence échoue
    raise ValueError("La volatilité implicite n'a pas convergé.")

def numerical_vega(S, K, T, r, sigma, option_type="call", epsilon=1e-5):
    """
    Approche numérique pour calculer la vega en utilisant les différences finies.
    """
    price_up = black_scholes(S, K, T, r, sigma + epsilon, option_type)
    price_down = black_scholes(S, K, T, r, sigma - epsilon, option_type)
    return (price_up - price_down) / (2 * epsilon)



In [ ]:
# Exemple d'utilisation de la fonction implied_volatility
S = 100       # Prix actuel de l'actif sous-jacent
K = 110       # Prix d'exercice
T = 1         # Temps jusqu'à l'échéance (en années)
r = 0.05      # Taux d'intérêt sans risque
market_price = 5  # Prix observé de l'option
option_type = "put"  # Type d'option ("call" ou "put")

# Calcul de la volatilité implicite
implied_vol, iter = implied_volatility(market_price, S, K, T, r, option_type)
print(f"Volatilité implicite : {implied_vol:.4f}")
print(f"Nombre d'itérations : {iter}")

In [ ]:
def simulate_black_scholes(S0, T, r, sigma, steps=10000):
    np.random.seed(42)  # Pour des résultats reproductibles
    dt = T / steps
    t = np.linspace(0, T, steps) 
    W = np.random.normal(0, np.sqrt(dt), steps)  # Incréments de Brownien de pas de temps dt 
    S = S0 * np.exp((r - 0.5 * sigma ** 2) * t + sigma * np.cumsum(W))
    return S, t

# incrémants indépendants de loi normale N(0,dt) plus restrictif que seulment Wt de loi normal N(0,t) où W n'est des lors plus un borwnien !


In [ ]:
# Exemple de simulation
S0 = 100  # Prix initial
T = 1     # Maturité (1 an)
r = 0.05  # Taux sans risque
sigma = 0.2  # Volatilité
trajectoire, temps = simulate_black_scholes(S0, T, r, sigma)

import matplotlib.pyplot as plt
plt.plot(temps, trajectoire)
plt.title("Simulation du modèle Black-Scholes")
plt.xlabel("Temps")
plt.ylabel("Prix de l'actif")
plt.show()


# Heston Method

In [5]:
def heston_monte_carlo(S0, v0, r, kappa, v_bar, gamma, rho, K, T, N, M):
    """
    S0 : float : Prix initial de l'actif sous-jacent
    v0 : float : Variance initiale
    r : float : Taux sans risque
    kappa : float : Vitesse de retour à la moyenne
    v_bar : float : Variance à long terme (moyenne)
    gamma : float : Volatilité de la variance
    rho : float : Corrélation entre les processus
    K : float : Prix d'exercice de l'option
    T : float : Temps jusqu'à maturité (en années)
    N : int : Nombre de pas de discrétisation
    M : int : Nombre de simulations de Monte-Carlo
    """
    dt = T / N  # Pas de temps
    
    # Initialisation des matrices pour S et v
    S = np.zeros((M, N + 1))
    v = np.zeros((M, N + 1))
    S[:, 0] = S0
    v[:, 0] = v0

    # Génération des variables normales
    for i in range(N):
        Z1 = np.random.normal(size=M)
        Z2 = np.random.normal(size=M)
        W1 = Z1
        W2 = rho * Z1 + np.sqrt(1 - rho**2) * Z2

        # Mise à jour de v (variance)
        v[:, i + 1] = v[:, i] 
        + kappa * (v_bar - v[:, i]) * dt + gamma * np.sqrt(np.maximum(v[:, i] * dt,0)) * W2
            

        # Mise à jour de S (prix de l'actif)
        S[:, i + 1] = S[:, i] * np.exp(
            (r - 0.5 * v[:, i]) * dt + np.sqrt(np.maximum(v[:, i] * dt,0)) * W1
        )

    # Calcul des payoffs
    payoffs = np.maximum(S[:, -1] - K, 0)

    # Estimation du prix de l'option
    C = np.exp(-r * T) * np.mean(payoffs)

    return C

In [6]:
# Exemple de paramètres
S0 = 100      # Prix initial de l'actif
r = 0.05      # Taux sans risque
K = 100       # Prix d'exercice
T = 1.0       # Temps jusqu'à maturité (1 an)

v0 = 0.04     # Variance initiale
kappa = 2.0   # Vitesse de retour à la moyenne
v_bar = 0.04  # Variance à long terme
gamma = 0.5   # Volatilité de la variance
rho = -0.7    # Corrélation entre les processus
N = 252       # Nombre de pas (1 par jour pour 1 an)
M = 10000     # Nombre de simulations Monte-Carlo

# Calcul du prix de l'option
prix_option = heston_monte_carlo(S0, v0, r, kappa, v_bar, gamma, rho, K, T, N, M)
print(f"Le prix de l'option call est : {prix_option:.2f}")

Le prix de l'option call est : 10.65


In [1]:
# # Fonction pour calculer d et g
# def calculate_d_and_g(s, kappa, gamma, rho):
#     d = np.sqrt((rho * gamma * 1j * s - kappa)**2 + (gamma**2) * (1j * s + s**2))
#     g = (kappa - rho * gamma * 1j * s - d) / (kappa - rho * gamma * 1j * s + d)
#     return d, g

# # Fonction f(s)
# def f(s):
#     d, g = calculate_d_and_g(s, kappa, gamma, rho)
    
#     term1 = np.exp(r * T) * S0**(1j * s)
#     term2 = (1 - g * np.exp(-d * T)) / (1 - g)
#     term3 = (2 * v_bar * kappa / gamma**2)
#     term4 = np.exp(
#         (v_bar * kappa * T / gamma**2) * (kappa - rho * gamma * 1j * s - d) 
#         + (v0 / gamma**2) *  (kappa - rho * gamma * 1j * s + d) * (np.exp(-d * T) - 1) / ( np.exp(-d * T) - g ) 
#     )

#     return term1 * term2 ** (-term3) * term4

# def midpoint_rule_transformed(func, n):
#     h = 1 / n  # Taille des sous-intervalles pour t dans [0, 1)
#     integral = 0
#     for i in range(n): # i va
#         t_mid = i * h + h / 2  # Point médian
#         x = t_mid / (1 - t_mid)  # Transformation
#         # print(x)
#         weight = 1 / (1 - t_mid)**2  # Jacobien
#         if i == n-1:
#             func(x)
#             print("s")
#         integral += func(x) * weight

#         # print(func(x))
#     return integral * h


In [12]:
# Intégration numérique pour le calcul du prix de l'option
# def heston_price(n):
#     integral_result = midpoint_rule_transformed(integrand, n)
#     return 1/2 * (S0 + K * np.exp(-r * T)) + (1 / np.pi) * integral_result
    

In [ ]:
# # Exemple avec transformation
# n_intervals = 1000  # Nombre de sous-intervalles
# result = heston_price(n_intervals)
# print(f"Résultat de l'intégrale avec la méthode du point médian (transformation) : {result}")

# Local Volatily Dupire method

In [2]:
def local_sigma(T, K, r, C):
    """
    Calcule σ(T, K) à partir des dérivées partielles du prix C(T, K).

    Paramètres :
    - T : maturité
    - K : strike
    - r : taux sans risque
    - C : fonction de prix C(T, K) l'interpolateur de prix

    Retourne :
    - σ_local(T, K)
    """ 
    
    # Première dérivée par rapport à T
    delta_T = 1e-4  # Pas pour la dérivée
    dC_dT = (C(T + delta_T, K) - C(T - delta_T, K)) / (2 * delta_T)

    # Première dérivée par rapport à K
    delta_K = 1e-4
    dC_dK = (C(T, K + delta_K) - C(T, K - delta_K)) / (2 * delta_K)

    # Deuxième dérivée par rapport à K
    d2C_dK2 = (C(T, K + delta_K) - 2 * C(T, K) + C(T, K - delta_K)) / (delta_K ** 2)

    # Calcul final de σ(T, K)
    numerator = dC_dT + r * K * dC_dK
    denominator = K**2 * d2C_dK2
    sigma_value = np.sqrt(2 * numerator / denominator)

    return sigma_value
 

In [6]:
import yfinance as yf
import pandas as pd
import numpy as np
from scipy.interpolate import CloughTocher2DInterpolator
import matplotlib.pyplot as plt
from datetime import datetime
# Ticker de l'action (par exemple, AAPL pour Apple)
ticker = "AAPL"

# Récupérer les options disponibles
stock = yf.Ticker(ticker) # Création de l'objet Ticker qui contient les informations sur l'action
option_dates = stock.options  # Les différentes maturités disponibles des options d'Apple
options_data = []

for option_date in option_dates:
    # Récupérer les chaînes d'options pour la date d'expiration spécifique
    options_chain = stock.option_chain(option_date)
    # Filtrer uniquement les calls et ajouter à la liste
    calls = options_chain.calls
    calls['maturity'] = option_date
    options_data.append(calls)


all_calls = pd.concat(options_data, ignore_index=True)
# Ajouter la colonne "maturity_years" en années à partir d'aujourd'hui
today = datetime.today()
all_calls['maturity_years'] = all_calls['maturity'].apply(
    lambda x: (datetime.strptime(x, "%Y-%m-%d") - today).days / 365
)

# Ne conserver que les colonnes nécessaires
all_calls = all_calls[['maturity_years', 'strike', 'lastPrice']]


# T array contenant les maturités 
T = np.array(all_calls['maturity_years'])
# print(T)
K = np.array(all_calls['strike'])
# print(K)
# faire un array avec les maturités et les strikes
points = np.array([T, K]).T 
values = np.array(all_calls['lastPrice'])

interpolator = CloughTocher2DInterpolator(points, values)

# Calcul de σ(T, K) pour un point donné
T_test = 1
K_test = 300
r = 0.02
sigma_value = local_sigma(T_test, K_test, r, interpolator)

print(f"Volatilité locale pour T={T_test}, K={K_test} : {sigma_value:.4f}")




Volatilité locale pour T=1, K=300 : 0.0274


Que faire une fois qu'on a calculé la volatilité locale ? 

https://quantipy.wordpress.com/2017/08/21/implementation-of-dupires-formula-for-local-volatilities/